# 지도를 통한 전국민 금융 스타일
지도를 활용하여 전국민의 금융 스타일을 시각화하는 것을 시도하였습니다.

### 패키지 로드

In [1]:
# 기본
import json
import pandas as pd
import os

In [2]:
# interactive widge을 위한 패키지
from IPython.display import display
from ipywidgets import interactive
from ipywidgets import widgets

In [3]:
# folium: 지도
import folium

In [4]:
# plotly: 그래프 패키지
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [5]:
from IPython.display import clear_output
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from folium.plugins import HeatMap , HeatMapWithTime, TimestampedGeoJson

### 데이터 로드

In [8]:
# 전국 시도 지리 데이터 다운로드. 
# geo_path = 'https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2013/json/skorea_provinces_geo_simple.json'
geo_path = '../../data/skorea_provinces_geo_simple.json'
geo_data = json.load(open(geo_path, encoding='utf-8'))

In [9]:
original_data = pd.read_csv("../../data/credit_card_data.csv")

In [10]:
# 'year'와 'month'를 합쳐 '2016/01' 형태의 값을 가지는 'time' 컬럼을 생성
tmp = original_data['month'].apply(str)
tmp = tmp.apply(lambda x: '0'+x  if len(x)==1 else x)
for index in original_data.index:
    original_data.loc[index, 'time'] = str(original_data.loc[index, 'year'])+'/'+str(tmp.loc[index])

In [11]:
# 지역에 매칭되는 코드
province_code = {'제주': '39',
 '경남': '38',
 '경북': '37',
 '전남': '36',
 '전북': '35',
 '충남': '34',
 '충북': '33',
 '강원': '32',
 '경기': '31',
 '울산': '26',
 '대전': '25',
 '광주': '24',
 '인천': '23',
 '대구': '22',
 '부산': '21',
 '서울': '11'}

In [12]:
province_rename = {'서울':'서울특별시',
                   '인천':'인천광역시',
                   '대전':'대전광역시',
                   '대구':'대구광역시',
                   '광주':'광주광역시',
                   '부산':'부산광역시',
                   '울산':'울산광역시',
                   '경기':'경기도',
                   '강원':'강원도',
                   '충북':'충청북도',
                   '충남':'충청남도',
                   '전북':'전라북도',
                   '전남':'전라남도',
                   '경북':'경상북도',
                   '경남':'경상남도',
                   '제주':'제주특별자치도'}

In [13]:
val_cols = ['num_opencard', 'num_usecard',
       'monthly_card_spend', 'monthly_lc', 'monthly_loan', 'monthly_bk_loan',
       'monthly_cd_loan', 'monthly_installments_loan',
       'monthly_insurance_loan', 'monthly_sbk_loan', 'loan_commitment',
       'inst_rep_loanb', 'ls_rep_loanb', 'credit_loan', 'mortgage_loan',
       'credit_card_payment', 'credit_card_installments_payment']

In [14]:
# 지역에 매칭되는 'code' 컬럼 생성
# '10대' 형태의 'ages' 컬럼을 int 10 형태로 변경
original_data['code'] = original_data['city'].replace(province_code)
original_data['ages'] = original_data.ages.apply(lambda x: int(x[:2]))

In [15]:
# 인구 수에 비례하도록 수치 값을 보정한 raw_num_opencard 컬럼 생성
for col in val_cols:
    original_data['raw_'+col] = (original_data[col]*original_data['population'])

In [16]:
# 지역정보가 있는 데이터는 성별정보가 없고, 그 반대의 경우도 마찬가지이므로
# 데이터의 중복을 피하기 위해 데이터를 두 가지로 나눔.
province_data = original_data[~original_data.city.isnull()]
sex_data = original_data[~original_data.sex.isnull()]

In [17]:
provinces = list(province_code.keys())
ages = [age for age in range(10,90+1, 10)]

In [18]:
province_data_tmp = province_data.groupby(['city']).sum()
province_data_tmp['city'] = province_data_tmp.index
province_data_tmp['code'] = province_data_tmp['city'].replace(province_code)

### 데이터 분석

### 시각화

In [19]:
timeline = []
for year in [2016, 2017]:
    for month in range(1, 12+1):
        if month < 10:
            month = '0'+str(month)
        timeline.append('{0}/{1}'.format(year, month))

In [20]:
# 시간 위젯
time_sliders = widgets.SelectionSlider(
    options=['전체']+timeline,
    description='시간',
    disabled=False,
    orientation='horizontal',
    readout=True
)
# 수치 계산 선택 위젯
metric_buttons = widgets.ToggleButtons(
    options=[('합계', 'sum'), ('평균', 1)],
    description='계산 방법:',
)
# 수치 위젯: 어떤 수치를 지도에 표시할 것인지 선택
val_buttons = widgets.Dropdown(
    options=val_cols,
    description='수치:',
)
# 신용점수 선택 위젯
options = range(720, 850+1, 10)
credit_sliders  = widgets.SelectionRangeSlider(
    options=options,
    index=(0,len(options)-1),
    description='신용점수',
    disabled=False
)
# 나이대 선택 위젯
options = range(10, 90+1, 10)
age_sliders = widgets.SelectionRangeSlider(
    options=options,
    index=(0,len(options)-1),
    description='연령대',
    disabled=False,
)

# 위젯을 container로 합침 
container = widgets.VBox([
                widgets.HBox([time_sliders]),
                widgets.HBox([metric_buttons]),
                widgets.HBox([val_buttons]),
                widgets.HBox([credit_sliders]),
                widgets.HBox([age_sliders]),
                ])

In [21]:
def draw_korea_map_val(df, col, metric):
    # 데이터 df를 받아 특정 컬럼 col을 
    # 합계 혹은 평균 metric으로 그룹화하여 지도에 색으로 값을 표시
    
    # 한국 전국 지도
    rome_lat, rome_lng = 36.2002, 127.054
    m = folium.Map(location=[rome_lat, rome_lng], zoom_start=7)
    
    # 합계 metric일 경우, city와 ages는 합하지 않고 그대로 유지
    if metric=='sum':
        tmp_df = df.groupby(['city'], as_index=False).mean()
        df = df.groupby(['city'], as_index=False).sum()
        df.loc[:, ['ages']] = tmp_df.loc[df.index, ['ages']]
        del tmp_df
    else: # metric=='mean':
        df = df.groupby(['city'], as_index=False).mean()
    
    df['code'] = df['city'].replace(province_code)
    
    # 표시 단위를 적절히 조정하기 위해 MinMaxScaler 적용하여 컬럼 col 값 업데이트
    tmp = df[col].values.reshape(-1,1)
    scaler = MinMaxScaler(feature_range=(0, 7))
    scaler.fit(tmp)
    df[col] = scaler.transform(tmp).reshape(-1)

    # geo_data의 key_on 값을 바탕으로 
    # data의 col 값을 색으로 표시한 것을 지도 m에 추가함. 
    folium.Choropleth(
        geo_data=geo_data,
        data = df,
        columns = ['code', col],
        key_on='feature.properties.code',
        fill_color='YlGnBu',
        legend_name=col,
        fill_opacity=0.5,
        line_opacity=0.5,
        #line_weight=2,
    ).add_to(m)

    return m #display(m)

In [22]:
def response(change):
    # 위젯에 변화가 감지되면 실행되는 코드
    
    # 이전 셀 아웃풋은 지움.
    clear_output()  
    
    temp_df = province_data.copy(deep=True) 
    
    # 시간
    if time_sliders.value != '전체':
        temp_df = temp_df[temp_df.time==time_sliders.value]

    # 신용 등급
    min_rat, max_rat = credit_sliders.value
    if min_rat==2 and max_rat==7:
        # not select
        None
    elif min_rat==max_rat:
        temp_df = temp_df[temp_df.avg_score==min_rate]
    else: # min_rat 초과, max_rat 미만
        temp_df = temp_df[(temp_df.avg_score>min_rat) & (temp_df.avg_score<=max_rat)]

    # 연령
    min_age, max_age = age_sliders.value
    if min_age==24 and max_age==99:
        # not select
        None
    elif min_age==max_age:
        temp_df = temp_df[temp_df.ages==min_age]
    else: # min_age 초과, max_age 미만
        temp_df = temp_df[(temp_df.ages>min_age) & (temp_df.ages<=max_age)]
    
    
    # 수치, 수치가 0인 row는 제외
    col = val_buttons.value
    temp_df = temp_df[temp_df[col]!=0]
    if col=='medium_resid_rat' or col=='large_resid_rat':
        temp_df.dropna(inplace=True)

    my_map = draw_korea_map_val(temp_df, col, metric_buttons.value)
    
    # 위젯 출력
    display(container)
    
    print ('선택된 그룹은 전체 그룹 중 {0:.1f}% ({1:d}명)를 차지합니다.'.format(len(temp_df)/len(province_data)*100, len(temp_df)))
    # 지도 출력
    display(my_map)

In [23]:
# 위젯 변화를 observe하여 response 함수를 실행
credit_sliders.observe(response, names="value")
val_buttons.observe(response, names="value")
age_sliders.observe(response, names="value")
metric_buttons.observe(response, names="value")
time_sliders.observe(response, names="value")

## 인터렉티브 데모
버튼 혹은 드랍다운 메뉴에서 항목을 선택할 경우, 선택한 집단에 따른 수치가 그래프로 표시됩니다.

In [24]:
# 위젯을 표시함. 
# 시간, 계산 방법, 수치, 신용점수, 연령대에 따라 수치가 어떻게 변하는지
# 한 눈에 알아볼 수 있도록 그래프 생성
display(container)
my_map = draw_korea_map_val(province_data, val_buttons.value, metric_buttons.value)
display(my_map)